In [1]:
from Bio import SeqIO, motifs
from Bio.Seq import Seq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import Counter
import shutil
import os
import csv
from io import StringIO
import random
import re
from Bio.SeqRecord import SeqRecord
import torch
from pycaret.classification import *
import json
import seaborn as sns

from sklearn.model_selection import train_test_split

In [ ]:
# store the result of one hot coding DNA as json dictionary
df = pd.read_csv("6_DNA_protein.csv")
df = df[df['Uniprot_ID'] != "P48743"]
df = df[df['Uniprot_ID'] != "Q53XW4"]
df.reset_index(drop=True, inplace=True)
alphabet = 'ATCGX'
one_hot_DNA = {}
for n in range(len(df)):
    input_string = df['DNA_Sequence'][n]
    char_to_index = {char: i for i, char in enumerate(alphabet)}
    one_hot_matrix = np.zeros((len(input_string), len(alphabet)),dtype=int)

    for i, char in enumerate(input_string):
        if char in char_to_index:
            one_hot_matrix[i, char_to_index[char]] = 1
    one_hot_matrix = one_hot_matrix.astype(str)
    
    chac_list = []
    for i in range(len(one_hot_matrix)):
        chac = ''
        for j in range(len(one_hot_matrix[i])):
            chac += one_hot_matrix[i][j]
        chac_list.append(chac)
    result = ''
    for a in range(len(chac_list)):
        result += chac_list[a]
    one_hot_DNA[input_string] = result
# Following part will store the one hot coding DNA into a json file.  
'''
with open('6_one_hot_encoding_DNA.json', 'w') as file:
    json.dump(one_hot_DNA, file)
'''

In [ ]:
# store the result of Word2Vec DNA as json dictionary
from gensim.models import Word2Vec
result = {}
for i in range(len(df)):
    temp_gen = []
    sequences = [df['DNA_Sequence'][i]]
    k_mer_size = 3
    k_mers = [''.join(seq[i:i+k_mer_size]) for seq in sequences for i in range(len(seq) - k_mer_size + 1)]

    model = Word2Vec([k_mers], vector_size=11, window=5, min_count=1, workers=4)
    for k_mer in k_mers:
        temp_gen +=  model.wv[k_mer].tolist()  
    result[sequences[0]] = temp_gen
# Following part will store the Word2Vec DNA into a json file.  
'''
with open('6_Word2Vec_encoding_DNA.json', 'w') as file:
    json.dump(result, file)
'''   

In [ ]:
# store the result of protein embedding as json dictionary
embeddings_0 = {}
embeddings_33 = {}

for file in file_name:
    a = torch.load('human_650m/' + file)
    embeddings_0[a['label'].split('|')[1]] = (a['mean_representations'][0].tolist())
    embeddings_33[a['label'].split('|')[1]] = (a['mean_representations'][0].tolist())
# Following part will store the protein embedding into a json file. 
'''
with open('6_protein_sequence.json', 'w') as file:
    json.dump(protein_dict, file)
'''